In [1]:
import pandas as pd
import requests
from pprint import pprint
import datetime
import json
import requests

In [2]:
from config import GOOGLE
from config import OWM

In [3]:
Overview=pd.read_csv("Overview.csv")
GP_list_all=Overview["AreaName"]
GP_list=GP_list_all.unique()
Final_GP_List=GP_list[2:len(GP_list)]
Asthma_data=Overview.loc[Overview['Indicator Name'] == "Asthma: QOF prevalence (6+ yrs)"][["AreaName","Count","Value"]]
Percentage=Asthma_data["Value"][2:len(GP_list)]

In [4]:
api_key=GOOGLE

In [5]:
gp_lat=[]
gp_lng=[]

url = "https://maps.googleapis.com/maps/api/place/textsearch/json?"

lat="52.478667"
long="-1.900024"

for gp in Final_GP_List:
    params={"key":api_key,
            'query':gp,
            'location':f"{lat},{long}",
            'radius':15000,
            'type':'health'
            }
    
    temp = requests.get(url, params=params).json()
    
    try:
        gp_lat.append(temp['results'][0]['geometry']['location']['lat'])
        gp_lng.append(temp['results'][0]['geometry']['location']['lng'])
    except IndexError:
        
        try:
            gp_lat.append(temp['results'][0]['geometry']['location']['lat'])
            gp_lng.append(temp['results'][0]['geometry']['location']['lng'])
            
            params={"key":api_key,
                'query':gp,
                'location':f"{lat},{long}",
                'radius':15000,
                'type':'doctor'
                }

            temp = requests.get(url, params=params).json()
        except IndexError:
            gp_lat.append("N/A")
            gp_lng.append("N/A")
    


In [7]:
df=pd.DataFrame(zip(Final_GP_List,gp_lat,gp_lng,Percentage),columns=["gp_practice","latitude","longitude","asthma_percentage"])
df.drop(gp_lng.index("N/A"),inplace=True)

In [8]:
GP_dict=[]
for row in df.iterrows():
    temp_dict={"practice_name":row[1][0],
    "latitude":row[1][1],
    "longitude":row[1][2],
    "asthma_percentage":row[1][3]}
    GP_dict.append(temp_dict)

In [9]:
GP_dict

[{'practice_name': 'Acocks Green Medical Centre',
  'latitude': 52.4500714,
  'longitude': -1.8301786,
  'asthma_percentage': 6.8472},
 {'practice_name': 'Alpha Medical Practice',
  'latitude': 52.4912121,
  'longitude': -1.8566404,
  'asthma_percentage': 7.9489},
 {'practice_name': 'Al-Shafa Medical Centre',
  'latitude': 52.5056399,
  'longitude': -1.8921101,
  'asthma_percentage': 6.545},
 {'practice_name': 'Alum Rock Medical Centre',
  'latitude': 52.4898917,
  'longitude': -1.8507548,
  'asthma_percentage': 7.4716},
 {'practice_name': 'Amaanah Medical Practice',
  'latitude': 52.4907265,
  'longitude': -1.8521147,
  'asthma_percentage': 6.4019},
 {'practice_name': 'Apollo Surgery',
  'latitude': 52.55443959999999,
  'longitude': -1.8924412,
  'asthma_percentage': 5.4482},
 {'practice_name': 'Arden Medical Centre',
  'latitude': 52.37942899999999,
  'longitude': -1.7431408,
  'asthma_percentage': 5.6651},
 {'practice_name': 'Arran Medical Centre',
  'latitude': 52.5032514,
  'longi

In [10]:
Weather_API=OWM

In [12]:
aqi=[]
co=[]
no=[]
no2=[]
o3=[]
so2=[]
pm2_5=[]
pm10=[]
nh3=[]

weather_url="http://api.openweathermap.org/data/2.5/air_pollution/history?"

for row in df.iterrows():
    
    params={"appid":Weather_API,
            'lat':row[1][1],
            'lon':row[1][2],
            'start':"1606266000",
            'end':"1637798400"
            }
    
    temp = requests.get(weather_url, params=params).json()
    
    avg1=0
    avg2=0
    avg3=0
    avg4=0
    avg5=0
    avg6=0
    avg7=0
    avg8=0
    avg9=0
    count=0
    for i in temp['list']:
        count+=1
        avg1=avg1+int(i['main']['aqi'])
        avg2=avg2+int(i['components']['co'])
        avg3=avg3+int(i['components']['no'])
        avg4=avg4+int(i['components']['no2'])
        avg5=avg5+int(i['components']['o3'])
        avg6=avg6+int(i['components']['so2'])
        avg7=avg7+int(i['components']['pm2_5'])
        avg8=avg8+int(i['components']['pm10'])
        avg9=avg9+int(i['components']['nh3'])
    
    aqi.append(avg1/count)
    co.append(avg2/count)
    no.append(avg3/count)
    no2.append(avg4/count)
    o3.append(avg5/count)
    so2.append(avg6/count)
    pm2_5.append(avg7/count)
    pm10.append(avg8/count)
    nh3.append(avg9/count)

In [14]:
df['aqi']=aqi
df['co']=co
df['no']=no
df['no2']=no2
df['o3']=o3
df['so2']=so2
df['pm2_5']=pm2_5
df['pm10']=pm10
df['nh3']=nh3

In [15]:
GP_dict=[]
for row in df.iterrows():
    temp_dict={"Practice Name":row[1][0],
    "Latitude":row[1][1],
    "Longitude":row[1][2],
    "Asthma Percentage":row[1][3],
    "aqi":row[1][4],
    "co":row[1][5],
    "no":row[1][6],
    "no2":row[1][7],
    "o3":row[1][8],
    "so2":row[1][9],
    "pm2_5":row[1][10],
    "pm10":row[1][11],
    "nh3":row[1][12]
              }
    GP_dict.append(temp_dict)

In [16]:
GP_dict

[{'Practice Name': 'Acocks Green Medical Centre',
  'Latitude': 52.4500714,
  'Longitude': -1.8301786,
  'Asthma Percentage': 6.8472,
  'aqi': 1.308379120879121,
  'co': 234.77861721611723,
  'no': 1.8444368131868132,
  'no2': 13.209478021978022,
  'o3': 49.76087454212454,
  'so2': 3.203525641025641,
  'pm2_5': 4.426282051282051,
  'pm10': 5.756181318681318,
  'nh3': 1.3494734432234432},
 {'Practice Name': 'Alpha Medical Practice',
  'Latitude': 52.4912121,
  'Longitude': -1.8566404,
  'Asthma Percentage': 7.9489,
  'aqi': 1.308379120879121,
  'co': 234.77861721611723,
  'no': 1.8444368131868132,
  'no2': 13.209478021978022,
  'o3': 49.76087454212454,
  'so2': 3.203525641025641,
  'pm2_5': 4.426282051282051,
  'pm10': 5.756181318681318,
  'nh3': 1.3494734432234432},
 {'Practice Name': 'Al-Shafa Medical Centre',
  'Latitude': 52.5056399,
  'Longitude': -1.8921101,
  'Asthma Percentage': 6.545,
  'aqi': 1.31856684981685,
  'co': 235.79246794871796,
  'no': 2.2152014652014653,
  'no2': 12

In [17]:
df

,gp_practice,latitude,longitude,asthma_percentage,aqi,co,no,no2,o3,so2,pm2_5,pm10,nh3
0,Acocks Green Medical Centre,52.450071,-1.830179,6.8472,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473
1,Alpha Medical Practice,52.491212,-1.85664,7.9489,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473
2,Al-Shafa Medical Centre,52.50564,-1.89211,6.5450,1.318567,235.792468,2.215201,12.828526,51.746223,3.257097,4.289263,5.570627,1.402587
3,Alum Rock Medical Centre,52.489892,-1.850755,7.4716,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473
4,Amaanah Medical Practice,52.490727,-1.852115,6.4019,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473
...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Woodland Road Surgery,52.410585,-1.961243,7.2904,1.303342,234.702610,2.070627,12.904075,49.847642,3.252404,4.359203,5.699176,1.397436
156,Wychall Lane Surgery,52.410103,-1.935609,7.4217,1.303342,234.702610,2.070627,12.904075,49.847642,3.252404,4.359203,5.699176,1.397436
157,Yardley Green Medical Centre,52.475873,-1.83905,7.8828,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473
158,Yardley Medical Centre,52.463084,-1.828371,4.0701,1.308379,234.778617,1.844437,13.209478,49.760875,3.203526,4.426282,5.756181,1.349473


In [18]:
GP_practice_df=[]

for row in df.iterrows():
    print(row[0])
    temp_df=[]
    aqi=[]
    co=[]
    no=[]
    no2=[]
    o3=[]
    so2=[]
    pm2_5=[]
    pm10=[]
    nh3=[]
    dt=[]
    GP=[]

    params={"appid":Weather_API,
            'lat':row[1][1],
            'lon':row[1][2],
            'start':"1606266000",
            'end':"1637798400"
            }
    
    temp = requests.get(weather_url, params=params).json()
    
    for i in temp['list']:
        aqi.append(i['main']['aqi'])
        co.append(i['components']['co'])
        no.append(i['components']['no'])
        no2.append(i['components']['no2'])
        o3.append(i['components']['o3'])
        so2.append(i['components']['so2'])
        pm2_5.append(i['components']['pm2_5'])
        pm10.append(i['components']['pm10'])
        nh3.append(i['components']['nh3'])
        dt.append(i['dt'])
        GP.append(row[1][0])
        
    temp_df=pd.DataFrame(zip(dt,aqi,co,no,no2,o3,so2,nh3,pm2_5,pm10,GP),columns=["date","air_quality_index","carbon_monoxide","nitrogen_monoxide","nitrogen_dioxide","ozone","sulphur_dioxide","ammonia","particultes_pm2_5","particulates_pm_10","practice_name"])
    
    GP_practice_df.append(temp_df)
    
GP_PRACTICE=GP_practice_df.pop(0)
for i in GP_practice_df:
    GP_PRACTICE=pd.concat([GP_PRACTICE,i])
    
GP_PRACTICE.reset_index(inplace=True)
GP_PRACTICE.drop('index', inplace=True, axis=1)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159


In [19]:
df.to_csv("gp_practice.csv",index=False)
GP_PRACTICE.to_csv("air_pollution_data.csv",index=False)

In [20]:
GP_PRACTICE

,date,air_quality_index,carbon_monoxide,nitrogen_monoxide,nitrogen_dioxide,ozone,sulphur_dioxide,ammonia,particultes_pm2_5,particulates_pm_10,practice_name
0,1606266000,1,208.62,0.00,3.68,54.36,1.12,0.44,4.87,7.20,Acocks Green Medical Centre
1,1606269600,1,205.28,0.00,3.13,53.64,1.04,0.39,4.58,6.32,Acocks Green Medical Centre
2,1606273200,1,203.61,0.00,2.87,52.21,1.01,0.37,4.08,5.33,Acocks Green Medical Centre
3,1606276800,1,203.61,0.00,2.91,50.78,1.07,0.34,3.60,4.51,Acocks Green Medical Centre
4,1606280400,1,205.28,0.00,3.38,47.92,1.18,0.36,3.17,3.82,Acocks Green Medical Centre
...,...,...,...,...,...,...,...,...,...,...,...
1389019,1637784000,2,300.41,2.51,50.04,3.89,7.75,0.95,4.62,6.22,Yardley Wood Health Centre
1389020,1637787600,2,310.42,1.16,50.04,8.49,9.66,0.86,5.01,6.39,Yardley Wood Health Centre
1389021,1637791200,1,287.06,0.07,30.16,24.32,9.89,0.66,3.48,4.30,Yardley Wood Health Centre
1389022,1637794800,1,283.72,0.04,21.76,28.61,8.46,0.56,2.64,3.23,Yardley Wood Health Centre
